# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
import pickle
from sqlalchemy import create_engine

In [29]:
# load data from database
engine = create_engine('sqlite:///Disaster-Response.db')
df = pd.read_sql_table('Disaster-Response', con=engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [30]:
def tokenize(text):
      # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
category_names = Y.columns
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i]), '------------------------------------------------------------------')

related :
             precision    recall  f1-score   support

          0       0.63      0.48      0.55      1553
          1       0.85      0.91      0.88      4954
          2       0.20      0.28      0.23        47

avg / total       0.79      0.80      0.79      6554
 ------------------------------------------------------------------
request :
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5388
          1       0.78      0.47      0.59      1166

avg / total       0.87      0.88      0.87      6554
 ------------------------------------------------------------------
offer :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6518
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6554
 ------------------------------------------------------------------
aid_related :
             precision    recall  f1-score   support

     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd462197268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
               criterion='gini', max_depth=None, max_features='auto',
               max_leaf_nodes=None, min_impurity_decrease=0.0,
               min_impurity_split=None, min_samples_leaf=1,
               min_samples_split=2, min_weight_fraction_leaf=0.0,
               n_estimators=10, n_jobs=1, oob_score=False, random_state

In [35]:
parameters = {
    "clf__estimator__min_samples_split":[2,3],
    "clf__estimator__min_samples_leaf": [1,2],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2);

In [36]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2 
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=3 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=3, total= 3.4min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=3 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total= 3.7min
[CV] clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=3 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total= 3.7min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2 
[CV]  clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2, total= 3.7min
[CV] clf__estimator__m

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 10.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 10.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__min_samples_split': [2, 3], 'clf__estimator__min_samples_leaf': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
y_pred = cv.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i]), '------------------------------------------------------------------')

related :
             precision    recall  f1-score   support

          0       0.65      0.48      0.55      1553
          1       0.85      0.92      0.88      4954
          2       0.24      0.30      0.26        47

avg / total       0.80      0.81      0.80      6554
 ------------------------------------------------------------------
request :
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5388
          1       0.79      0.47      0.59      1166

avg / total       0.88      0.88      0.87      6554
 ------------------------------------------------------------------
offer :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6518
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6554
 ------------------------------------------------------------------
aid_related :
             precision    recall  f1-score   support

     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [41]:
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier

pipeline2  = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state=0))))
    ])

pipeline2.fit(X_train, Y_train)
y_pred = pipeline2.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], y_pred[:,i]), '------------------------------------------------------------------')

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


related :
             precision    recall  f1-score   support

          0       0.64      0.54      0.59      1553
          1       0.86      0.91      0.88      4954
          2       0.87      0.28      0.42        47

avg / total       0.81      0.82      0.81      6554
 ------------------------------------------------------------------
request :
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5388
          1       0.78      0.59      0.67      1166

avg / total       0.89      0.90      0.89      6554
 ------------------------------------------------------------------
offer :
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6518
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6554
 ------------------------------------------------------------------
aid_related :
             precision    recall  f1-score   support

     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [42]:
with open('ML-Disaster.pkl', 'wb') as file:
    pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.